In [1]:
import pandas as pd
import numpy as np

# Read in the data

## Read in joke id and joke text
jester_joke_data = pd.read_csv('jester_items.csv')

## Read in joke ratings
jester_joke_ratings = pd.read_csv('jester_ratings.csv')


In [11]:
## Read reddit jokes and unify the format for the joke data
reddit_jokes = pd.read_json('reddit_jokes.json')
reddit_jokes['jokeId'] = reddit_jokes.index

## TODO: Add this dataframe to the jester_joke_data dataframe and update the index
reddit_jokes['jokeText'] = reddit_jokes['title'] + '\n' + reddit_jokes['body']
reddit_joke_data = reddit_jokes[['jokeId', 'jokeText']]

In [23]:
## Read stupidstuff.json and unify the format for the joke data
stupidstuff_jokes = pd.read_json('stupidstuff.json')

stupidstuff_jokes['jokeId'] = stupidstuff_jokes['id']
stupidstuff_jokes['jokeText'] = stupidstuff_jokes['body']

stupidstuff_joke_data = stupidstuff_jokes[['jokeId', 'jokeText']]

In [43]:
print(jester_joke_data.columns.value_counts() == reddit_joke_data.columns.value_counts())
print(jester_joke_data.columns.value_counts() == stupidstuff_joke_data.columns.value_counts())

jokeId      True
jokeText    True
dtype: bool
jokeId      True
jokeText    True
dtype: bool


In [46]:
## Add all the data sets into one and update the index

joke_data = pd.concat([jester_joke_data, reddit_joke_data, stupidstuff_joke_data], ignore_index=True)

## Since we don't need the joke ratings for any of them, we don't need to combine them yet

joke_ratings_dataset = pd.DataFrame(columns=['jokeID', 'user_id', 'rating'])

,jokeId,jokeText
0,1,A blackjack dealer and a player with a thirtee...
1,2,"At a dinner party, several of the guests were ..."
2,3,One day this cop pulls over a blonde for speed...
3,4,Three women are about to be executed for crime...
4,5,A girl came skipping home FROM school one day....
...,...,...
3768,3769,
3769,3770,The Pope and the Queen of England are on the s...
3770,3771,
3771,3772,Letter to Xerox and the Reply\n\nDear Kings of...


In [137]:
# Focusing on the first few users for now

joke_ratings = joke_ratings[joke_ratings['userId'] < 5]

In [139]:
# We create new columns for all the jokes accoring to the user ratings

for index, row in joke_ratings.iterrows():
    # Create new column if it doesn't exist
    if 'user_' + str(row['userId']) + '_rating' not in joke_ratings.columns:
        joke_ratings['user_' + str(row['userId']) + '_rating'] = np.nan
    joke_ratings['user_' + str(row['userId']) + '_rating'][index] = row['rating']

/var/folders/ps/gtpc0wkx1j12zfc6gjn0kwsw0000gn/T/ipykernel_33692/2125598061.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joke_ratings['user_' + str(row['userId']) + '_rating'][index] = row['rating']
/var/folders/ps/gtpc0wkx1j12zfc6gjn0kwsw0000gn/T/ipykernel_33692/2125598061.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joke_ratings['user_' + str(row['userId']) + '_rating'][index] = row['rating']
/var/folders/ps/gtpc0wkx1j12zfc6gjn0kwsw0000gn/T/ipykernel_33692/2125598061.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [140]:
# We create new columns for all the jokes accoring to the user ratings

for index, row in joke_ratings.iterrows():
    for col in joke_ratings.columns:
        if 'user_' in col and pd.isna(row[col]):
            user_num = int(float(col.split('_')[1]))
            joke_id = int(row['jokeId'])
            user_rating = joke_ratings[(joke_ratings['userId'] == user_num) & (joke_ratings['jokeId'] == joke_id)]['rating']
            # joke_ratings[col][index] = user_rating
            if user_rating.values:
                joke_ratings[col][index] = user_rating
            else:
                joke_ratings[col][index] = 0
            

/var/folders/ps/gtpc0wkx1j12zfc6gjn0kwsw0000gn/T/ipykernel_33692/3825535566.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joke_ratings[col][index] = user_rating
/var/folders/ps/gtpc0wkx1j12zfc6gjn0kwsw0000gn/T/ipykernel_33692/3825535566.py:10: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if user_rating.values:
/var/folders/ps/gtpc0wkx1j12zfc6gjn0kwsw0000gn/T/ipykernel_33692/3825535566.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joke_ratings[col][index] = 0


In [141]:
joke_ratings[joke_ratings['jokeId'] == 5]

,userId,jokeId,rating,user_1.0_rating,user_2.0_rating,user_3.0_rating,user_4.0_rating
0,1,5,0.219,0.219,-9.688,-9.844,-5.812
62,2,5,-9.688,0.219,-9.688,-9.844,-5.812
96,3,5,-9.844,0.219,-9.688,-9.844,-5.812
114,4,5,-5.812,0.219,-9.688,-9.844,-5.812


In [151]:
joke_ratings.drop_duplicates(subset=['jokeId'], keep='first', inplace=True)

In [154]:
joke_ratings.drop(columns=['userId', 'rating'], inplace=True)

In [ ]:
# We create new columns for all the jokes accoring to the user ratings

for index, row in joke_ratings.iterrows():
    joke_id = row['jokeId']

    
    # Populate the new columns with the ratings
    joke_ratings['user_' + str(row['userId']) + '_rating'][index] 

In [49]:
# We drop the userId and rating columns

joke_ratings = joke_ratings.drop(['userId', 'rating'], axis=1)

In [50]:
# # We checkand delete for duplicate rows

# joke_ratings = joke_ratings.drop_duplicates()

In [155]:
joke_ratings

,jokeId,user_1.0_rating,user_2.0_rating,user_3.0_rating,user_4.0_rating
0,5,0.219,-9.688,-9.844,-5.812
1,7,-9.281,9.938,-9.844,-4.500
2,8,-9.281,9.531,-7.219,-4.906
3,13,-6.781,9.938,-2.031,0.000
4,15,0.875,0.406,-9.938,0.000
...,...,...,...,...,...
95,48,0.000,9.281,0.000,0.000
106,33,0.000,0.000,-9.812,0.000
108,37,0.000,0.000,-9.812,0.000
109,38,0.000,0.000,0.062,0.000
